In [1]:
import os
from shutil import copy

#import tensorflow as tf
#import tensorflow.keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, MaxPooling2D, Conv2D, Input, BatchNormalization
from keras.regularizers import l1
from keras.optimizers import RMSprop, Adagrad, Adam, SGD
from keras.applications import VGG16
from keras.applications.mobilenetv2 import MobileNetV2
from keras.models import load_model, Model
from keras.callbacks import ModelCheckpoint

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import confusion_matrix

import PIL.Image

from pprint import pprint

C:\Users\Sophie\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Getting Started

In [48]:
def applyBoundingBox(image, boundingbox):
    x = int(float(boundingbox[0]))
    y = int(float(boundingbox[1]))
    width = int(float(boundingbox[2]))
    height = int(float(boundingbox[3]))
    #print([x, y, width, height])
        
    imagecropped = image[y:y+height,x:x+width]
    
#     print("len(image): ", len(image))
#     print("len(image)[0]: ", len(image[0]))
#     print("len(image)[0][0]: ", len(image[0][0]))
    
#     print("len(imagecropped): ", len(imagecropped))
#     print("len(imagecropped)[0]: ", len(imagecropped[0]))
#     print("len(imagecropped)[0][0]: ", len(imagecropped[0][0]))
    return imagecropped

In [45]:
def copyImage(src, label, tr_te, dst="", image=None):
    
        cat = os.path.join(tr_te, label)
        new_dst = os.path.join(dst,cat)
        
        #print(src)
        #print(cat)
        #print(new_dst)
        
        # check if directory exists
        if os.path.isdir(new_dst):
            pass
        else:        
            # make diretory
            os.mkdir(new_dst)

        #print(new_dst)
        if image is not None:
            file = src.split("/")[-1]
            new_dst = os.path.join(new_dst,file)
            result = PIL.Image.fromarray(image)
            result.save(new_dst)
        else:
            copy(src, new_dst)

In [49]:
#generateData("./CUB_200_2011/CUB_200_2011/CUB_200_2011/train.txt", "./Outex_TC_00030.tar/Outex-TC-00030/images/", "C:/Users/Sophie/Documents/GitHub/CSI5139/csi5139_a2/Outex-TC-00030/test/")

drt = "./CUB_200_2011/CUB_200_2011/CUB_200_2011/"
drt_images = "./CUB_200_2011/CUB_200_2011/CUB_200_2011/images"
dst = "./CUB_200_2011_generator/"
dst_bb = "./CUB_200_2011_generator_boundingbox/"

# images.txt
with open(os.path.join(drt, "images.txt"), "r", encoding='utf-8') as f:
    images_txt = f.read().splitlines()

# train_test_split.txt
with open(os.path.join(drt, "train_test_split.txt"), "r", encoding='utf-8') as f:
    tr_te_txt = f.read().splitlines()

# image_class_labels.txt
with open(os.path.join(drt, "image_class_labels.txt"), "r", encoding='utf-8') as f:
    class_labels_txt = f.read().splitlines()

# bounding_boxes.txt
with open(os.path.join(drt, "bounding_boxes.txt"), "r", encoding='utf-8') as f:
    bounding_boxes_txt = f.read().splitlines()

print(images_txt[:2])
print(tr_te_txt[:2])
print(class_labels_txt[:2])
print(bounding_boxes_txt[:2])
print("# images_txt: ", len(images_txt))
print("# tr_te_txt: ", len(tr_te_txt))
print("# class_labels_txt: ", len(class_labels_txt))
print("# bounding_boxes_txt: ", len(bounding_boxes_txt))

#birdspecies_subset = {"cardinal": 17, "american_goldﬁnch": 47, "evening_grosbeak": 55, "song_sparrow": 129, "tree_sparrow": 130, "house_sparrow": 118, "fox_sparrow":120, "canada_warbler":162, "myrtle warbler": 171, "house_wren": 196 , "red-winged_blackbird": 10, "boat_tailed_grackle":49}
birdspecies_subset = {"17": "cardinal", "47": "american_goldﬁnch", "55": "evening_grosbeak", "129": "song_sparrow", "130": "tree_sparrow", "118": "house_sparrow", "120": "fox_sparrow", "162": "canada_warbler", "171": "myrtle warbler", "196": "house_wren", "10": "red-winged_blackbird", "49": "boat_tailed_grackle"}
label_subset = set(birdspecies_subset.keys())
print("label_subset: ", label_subset)

boundingbox = True
for i, label in enumerate(class_labels_txt):
    label = label.split(' ')[-1]
    if label in label_subset:
        file = images_txt[i].split(' ')[-1]
        tr_te = tr_te_txt[i].split(' ')[-1]
        b_b = bounding_boxes_txt[i].split(' ')[1:]

        if tr_te == "1":
            tr_te = "train"
        elif tr_te == "0":
            tr_te = "test"
        else:
            break
        
        print(file)
        src = os.path.join(drt_images, file)
        
        if boundingbox:
            image = PIL.Image.open(src)
            image = np.array(image)
            image = applyBoundingBox(image, b_b)
            copyImage(file, label, tr_te, dst_bb, image)
        else:
            src = os.path.join(drt_images, file)
            copyImage(src, label, tr_te, dst) 

['1 001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg', '2 001.Black_footed_Albatross/Black_Footed_Albatross_0009_34.jpg']
['1 0', '2 1']
['1 1', '2 1']
['1 60.0 27.0 325.0 304.0', '2 139.0 30.0 153.0 264.0']
# images_txt:  11788
# tr_te_txt:  11788
# class_labels_txt:  11788
# bounding_boxes_txt:  11788
label_subset:  {'196', '55', '162', '129', '17', '130', '118', '47', '120', '171', '10', '49'}
010.Red_winged_Blackbird/Red_Winged_Blackbird_0091_4096.jpg
010.Red_winged_Blackbird/Red_Winged_Blackbird_0059_4612.jpg
010.Red_winged_Blackbird/Red_Winged_Blackbird_0054_4625.jpg
010.Red_winged_Blackbird/Red_Winged_Blackbird_0022_4483.jpg
010.Red_winged_Blackbird/Red_Winged_Blackbird_0017_4116.jpg
010.Red_winged_Blackbird/Red_Winged_Blackbird_0079_4527.jpg
010.Red_winged_Blackbird/Red_Winged_Blackbird_0075_4953.jpg
010.Red_winged_Blackbird/Red_Winged_Blackbird_0005_5636.jpg
010.Red_winged_Blackbird/Red_Winged_Blackbird_0058_4141.jpg
010.Red_winged_Blackbird/Red_Winged_Blackbird_0

047.American_Goldfinch/American_Goldfinch_0131_32911.jpg
047.American_Goldfinch/American_Goldfinch_0046_32105.jpg
047.American_Goldfinch/American_Goldfinch_0107_32618.jpg
047.American_Goldfinch/American_Goldfinch_0022_32111.jpg
047.American_Goldfinch/American_Goldfinch_0129_32625.jpg
047.American_Goldfinch/American_Goldfinch_0064_32142.jpg
047.American_Goldfinch/American_Goldfinch_0089_32152.jpg
047.American_Goldfinch/American_Goldfinch_0014_32154.jpg
047.American_Goldfinch/American_Goldfinch_0056_32169.jpg
047.American_Goldfinch/American_Goldfinch_0106_32182.jpg
047.American_Goldfinch/American_Goldfinch_0100_32183.jpg
047.American_Goldfinch/American_Goldfinch_0122_32186.jpg
047.American_Goldfinch/American_Goldfinch_0118_32210.jpg
047.American_Goldfinch/American_Goldfinch_0121_32725.jpg
047.American_Goldfinch/American_Goldfinch_0029_32218.jpg
047.American_Goldfinch/American_Goldfinch_0026_32222.jpg
047.American_Goldfinch/American_Goldfinch_0103_32225.jpg
047.American_Goldfinch/American

130.Tree_Sparrow/Tree_Sparrow_0100_122945.jpg
130.Tree_Sparrow/Tree_Sparrow_0044_122946.jpg
130.Tree_Sparrow/Tree_Sparrow_0005_122949.jpg
130.Tree_Sparrow/Tree_Sparrow_0097_123462.jpg
130.Tree_Sparrow/Tree_Sparrow_0032_123489.jpg
130.Tree_Sparrow/Tree_Sparrow_0011_122964.jpg
130.Tree_Sparrow/Tree_Sparrow_0112_125014.jpg
130.Tree_Sparrow/Tree_Sparrow_0072_123991.jpg
130.Tree_Sparrow/Tree_Sparrow_0042_124512.jpg
130.Tree_Sparrow/Tree_Sparrow_0022_123496.jpg
130.Tree_Sparrow/Tree_Sparrow_0041_123497.jpg
130.Tree_Sparrow/Tree_Sparrow_0062_123000.jpg
130.Tree_Sparrow/Tree_Sparrow_0076_123669.jpg
130.Tree_Sparrow/Tree_Sparrow_0043_124034.jpg
130.Tree_Sparrow/Tree_Sparrow_0015_125105.jpg
130.Tree_Sparrow/Tree_Sparrow_0018_123574.jpg
130.Tree_Sparrow/Tree_Sparrow_0095_124090.jpg
130.Tree_Sparrow/Tree_Sparrow_0125_123078.jpg
130.Tree_Sparrow/Tree_Sparrow_0058_123591.jpg
130.Tree_Sparrow/Tree_Sparrow_0101_124104.jpg
130.Tree_Sparrow/Tree_Sparrow_0119_124114.jpg
130.Tree_Sparrow/Tree_Sparrow_0080

196.House_Wren/House_Wren_0055_187397.jpg
196.House_Wren/House_Wren_0071_187399.jpg
196.House_Wren/House_Wren_0083_187406.jpg
196.House_Wren/House_Wren_0087_187946.jpg
196.House_Wren/House_Wren_0061_187911.jpg
196.House_Wren/House_Wren_0004_187448.jpg
196.House_Wren/House_Wren_0012_187973.jpg
196.House_Wren/House_Wren_0103_187466.jpg
196.House_Wren/House_Wren_0046_187477.jpg
196.House_Wren/House_Wren_0064_187489.jpg
196.House_Wren/House_Wren_0117_187492.jpg
196.House_Wren/House_Wren_0005_187493.jpg
196.House_Wren/House_Wren_0016_188006.jpg
196.House_Wren/House_Wren_0114_188016.jpg
196.House_Wren/House_Wren_0057_187157.jpg
196.House_Wren/House_Wren_0047_188036.jpg
196.House_Wren/House_Wren_0091_188046.jpg
196.House_Wren/House_Wren_0001_188047.jpg
196.House_Wren/House_Wren_0011_187547.jpg
196.House_Wren/House_Wren_0119_187552.jpg
196.House_Wren/House_Wren_0063_188076.jpg
196.House_Wren/House_Wren_0134_188077.jpg
196.House_Wren/House_Wren_0116_187604.jpg
196.House_Wren/House_Wren_0123_187

In [20]:
datagen = ImageDataGenerator(rescale=.1/255)

train_batchsize = 4 # 160 images per 68 classes
test_batchsize = 4 # 20 images per 68 classes

train_generator = datagen.flow_from_directory(
    directory=r"./CUB_200_2011_generator_boundingbox/train/",
    target_size=(96, 96),
    color_mode="rgb",
    batch_size=train_batchsize,
    class_mode="categorical",
    #shuffle=True,
    seed=42
)

test_generator = datagen.flow_from_directory(
    directory=r"./CUB_200_2011_generator_boundingbox/test/",
    target_size=(96, 96),
    color_mode="rgb",
    batch_size=test_batchsize,
    class_mode="categorical",
    #shuffle=True,
    seed=42
)

print("Train generator length: ", len(train_generator))
print("Test generator length: ", len(test_generator))
print("Target size: ", train_generator.target_size)
#print("Generator indices: ", train_generator.class_indices)
#print("Generator filenames:", train_generator.filenames[:10])

Found 359 images belonging to 12 classes.
Found 357 images belonging to 12 classes.
Train generator length:  90
Test generator length:  90
Target size:  (96, 96)


In [3]:
#input_tensor = Input(shape=(96, 96, 3))
#mobilenetv2_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(96, 96, 3), input_tensor=input_tensor)
mobilenetv2_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(96, 96, 3))

In [21]:
epochs = 10

#mobilenetv2_model.summary()
for layer in mobilenetv2_model.layers:
    layer.trainable = False

# last layers output
x = mobilenetv2_model.layers[-1].output

# Flatten as before
x = Flatten()(x)
#x = Dropout(0.5)(x)
x = Dense(8, activation='relu')(x)

# Apply three different regularizations
x_dropout = Dropout(0.2)(x)
x_batchnorm = BatchNormalization()(x)

dropout = Dense(12, activation='softmax')(x_dropout)
batchnorm = Dense(12, activation='softmax')(x_batchnorm)
l1reg = Dense(12, activation='softmax', kernel_regularizer=l1(0.1), activity_regularizer=l1(0.1))(x)

mv2_dropout_transfer = Model(inputs=mobilenetv2_model.input, outputs=dropout)
mv2_batchnorm_transfer = Model(inputs=mobilenetv2_model.input, outputs=batchnorm)
mv2_l1reg_transfer = Model(inputs=mobilenetv2_model.input, outputs=l1reg)

sgd = SGD()
cp_callback_dropout = ModelCheckpoint(filepath="./runs/mobilenetv2_dropout.ckpt", verbose=1, save_best_only=True)
cp_callback_batchnorm = ModelCheckpoint(filepath="./runs/mobilenetv2_batchnorm.ckpt", save_weights_only=True, verbose=1, save_best_only=True)
cp_callback_l1reg = ModelCheckpoint(filepath="./runs/mobilenetv2_l1reg.ckpt", save_weights_only=True, verbose=1, save_best_only=True)

mv2_dropout_transfer.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
mv2_batchnorm_transfer.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
mv2_l1reg_transfer.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
for layer in mv2_dropout_transfer.layers:
    print(layer, layer.trainable)
    
mv2_dropout_transfer.summary()

hist_mv2_dropout = mv2_dropout_transfer.fit_generator(
        train_generator,
        steps_per_epoch=(train_generator.samples/train_generator.batch_size),
        epochs=epochs,
        validation_data=test_generator,
        validation_steps=(test_generator.samples/test_generator.batch_size),
        verbose=1)

<keras.engine.input_layer.InputLayer object at 0x0000025F6B17B400> False
<keras.layers.convolutional.Conv2D object at 0x0000025F6B17BA90> False
<keras.layers.normalization.BatchNormalization object at 0x0000025F6B17B9B0> False
<keras.layers.advanced_activations.ReLU object at 0x0000025F6B17BC50> False
<keras.layers.convolutional.DepthwiseConv2D object at 0x0000025F6B226C50> False
<keras.layers.normalization.BatchNormalization object at 0x0000025F6B17B630> False
<keras.layers.advanced_activations.ReLU object at 0x0000025F6B2C2A90> False
<keras.layers.convolutional.Conv2D object at 0x0000025F6B2D4710> False
<keras.layers.normalization.BatchNormalization object at 0x0000025F70F3D7B8> False
<keras.layers.convolutional.Conv2D object at 0x0000025F70F0B828> False
<keras.layers.normalization.BatchNormalization object at 0x0000025F70FC7908> False
<keras.layers.advanced_activations.ReLU object at 0x0000025F70FFC630> False
<keras.layers.convolutional.DepthwiseConv2D object at 0x0000025F7109E940> 

Epoch 1/10
30/29 [==============================] - 84s 3s/step - loss: 2.4139 - acc: 0.1950 - val_loss: 2.8578 - val_acc: 0.0840
Epoch 2/10
30/29 [==============================] - 158s 5s/step - loss: 1.7330 - acc: 0.4038 - val_loss: 2.7724 - val_acc: 0.0868
Epoch 3/10
30/29 [==============================] - 155s 5s/step - loss: 1.2721 - acc: 0.5571 - val_loss: 3.3821 - val_acc: 0.0896
Epoch 4/10
30/29 [==============================] - 161s 5s/step - loss: 0.8531 - acc: 0.7328 - val_loss: 3.7322 - val_acc: 0.0840
Epoch 5/10
29/29 [============================>.] - ETA: 1s - loss: 0.7106 - acc: 0.7665

KeyboardInterrupt: 

In [ ]:
# 4 batch, 8 dense
for layer in mv2_batchnorm_transfer.layers:
    print(layer, layer.trainable)
    
mv2_batchnorm_transfer.summary()

hist_mv2_batchnorm = mv2_batchnorm_transfer.fit_generator(
        train_generator,
        steps_per_epoch=(train_generator.samples/train_generator.batch_size),
        epochs=epochs,
        validation_data=test_generator,
        validation_steps=(test_generator.samples/test_generator.batch_size),
        verbose=1)

<keras.engine.input_layer.InputLayer object at 0x000001F98DC9AF28> False
<keras.layers.convolutional.Conv2D object at 0x000001F98DC9A7B8> False
<keras.layers.normalization.BatchNormalization object at 0x000001F98DC9B518> False
<keras.layers.advanced_activations.ReLU object at 0x000001F98DCACF60> False
<keras.layers.convolutional.DepthwiseConv2D object at 0x000001F9889E3208> False
<keras.layers.normalization.BatchNormalization object at 0x000001F98DD677B8> False
<keras.layers.advanced_activations.ReLU object at 0x000001F98DD67828> False
<keras.layers.convolutional.Conv2D object at 0x000001F98DDE2B00> False
<keras.layers.normalization.BatchNormalization object at 0x000001F993A60E80> False
<keras.layers.convolutional.Conv2D object at 0x000001F993A307B8> False
<keras.layers.normalization.BatchNormalization object at 0x000001F993AFBA58> False
<keras.layers.advanced_activations.ReLU object at 0x000001F993AC1668> False
<keras.layers.convolutional.DepthwiseConv2D object at 0x000001F993B1D4E0> 

Epoch 1/10
90/89 [==============================] - 95s 1s/step - loss: 2.4278 - acc: 0.1787 - val_loss: 2.6072 - val_acc: 0.1036
Epoch 2/10
90/89 [==============================] - 85s 949ms/step - loss: 2.0095 - acc: 0.3417 - val_loss: 2.5918 - val_acc: 0.0840
Epoch 3/10
90/89 [==============================] - 89s 984ms/step - loss: 1.8063 - acc: 0.4620 - val_loss: 2.5814 - val_acc: 0.0924
Epoch 4/10
90/89 [==============================] - 85s 940ms/step - loss: 1.7001 - acc: 0.4546 - val_loss: 2.6457 - val_acc: 0.0980
Epoch 5/10
90/89 [==============================] - 84s 935ms/step - loss: 1.6680 - acc: 0.4833 - val_loss: 2.6882 - val_acc: 0.0896
Epoch 6/10
89/89 [============================>.] - ETA: 0s - loss: 1.4784 - acc: 0.5421

In [8]:
# 8 batch, 16 dense
for layer in mv2_batchnorm_transfer.layers:
    print(layer, layer.trainable)
    
mv2_batchnorm_transfer.summary()

hist_mv2_batchnorm = mv2_batchnorm_transfer.fit_generator(
        train_generator,
        steps_per_epoch=(train_generator.samples/train_generator.batch_size),
        epochs=epochs,
        validation_data=test_generator,
        validation_steps=(test_generator.samples/test_generator.batch_size),
        verbose=1)

<keras.engine.input_layer.InputLayer object at 0x000001F98DC9AF28> False
<keras.layers.convolutional.Conv2D object at 0x000001F98DC9A7B8> False
<keras.layers.normalization.BatchNormalization object at 0x000001F98DC9B518> False
<keras.layers.advanced_activations.ReLU object at 0x000001F98DCACF60> False
<keras.layers.convolutional.DepthwiseConv2D object at 0x000001F9889E3208> False
<keras.layers.normalization.BatchNormalization object at 0x000001F98DD677B8> False
<keras.layers.advanced_activations.ReLU object at 0x000001F98DD67828> False
<keras.layers.convolutional.Conv2D object at 0x000001F98DDE2B00> False
<keras.layers.normalization.BatchNormalization object at 0x000001F993A60E80> False
<keras.layers.convolutional.Conv2D object at 0x000001F993A307B8> False
<keras.layers.normalization.BatchNormalization object at 0x000001F993AFBA58> False
<keras.layers.advanced_activations.ReLU object at 0x000001F993AC1668> False
<keras.layers.convolutional.DepthwiseConv2D object at 0x000001F993B1D4E0> 

45/44 [==============================] - 109s 2s/step - loss: 0.5448 - acc: 0.8833 - val_loss: 2.6309 - val_acc: 0.1401
Epoch 2/10
45/44 [==============================] - 110s 2s/step - loss: 0.4618 - acc: 0.9191 - val_loss: 2.6363 - val_acc: 0.1569
Epoch 3/10
45/44 [==============================] - 109s 2s/step - loss: 0.4924 - acc: 0.9302 - val_loss: 2.6569 - val_acc: 0.1485
Epoch 4/10
45/44 [==============================] - 110s 2s/step - loss: 0.3775 - acc: 0.9385 - val_loss: 2.6937 - val_acc: 0.1457
Epoch 5/10
45/44 [==============================] - 108s 2s/step - loss: 0.4281 - acc: 0.9389 - val_loss: 2.7432 - val_acc: 0.1569
Epoch 6/10
45/44 [==============================] - 109s 2s/step - loss: 0.3565 - acc: 0.9218 - val_loss: 2.7164 - val_acc: 0.1457
Epoch 7/10
45/44 [==============================] - 115s 3s/step - loss: 0.3788 - acc: 0.9333 - val_loss: 2.7921 - val_acc: 0.1569
Epoch 8/10
45/44 [==============================] - 116s 3s/step - loss: 0.3383 - acc: 0.9417 

In [6]:
#4 batch, 16 dense
for layer in mv2_batchnorm_transfer.layers:
    print(layer, layer.trainable)
    
mv2_batchnorm_transfer.summary()

hist_mv2_batchnorm = mv2_batchnorm_transfer.fit_generator(
        train_generator,
        steps_per_epoch=(train_generator.samples/train_generator.batch_size),
        epochs=epochs,
        validation_data=test_generator,
        validation_steps=(test_generator.samples/test_generator.batch_size),
        verbose=1)

<keras.engine.input_layer.InputLayer object at 0x000001F98DC9AF28> False
<keras.layers.convolutional.Conv2D object at 0x000001F98DC9A7B8> False
<keras.layers.normalization.BatchNormalization object at 0x000001F98DC9B518> False
<keras.layers.advanced_activations.ReLU object at 0x000001F98DCACF60> False
<keras.layers.convolutional.DepthwiseConv2D object at 0x000001F9889E3208> False
<keras.layers.normalization.BatchNormalization object at 0x000001F98DD677B8> False
<keras.layers.advanced_activations.ReLU object at 0x000001F98DD67828> False
<keras.layers.convolutional.Conv2D object at 0x000001F98DDE2B00> False
<keras.layers.normalization.BatchNormalization object at 0x000001F993A60E80> False
<keras.layers.convolutional.Conv2D object at 0x000001F993A307B8> False
<keras.layers.normalization.BatchNormalization object at 0x000001F993AFBA58> False
<keras.layers.advanced_activations.ReLU object at 0x000001F993AC1668> False
<keras.layers.convolutional.DepthwiseConv2D object at 0x000001F993B1D4E0> 

Epoch 1/10
90/89 [==============================] - 116s 1s/step - loss: 2.2884 - acc: 0.2333 - val_loss: 2.7175 - val_acc: 0.1064
Epoch 2/10
90/89 [==============================] - 176s 2s/step - loss: 1.7936 - acc: 0.4898 - val_loss: 2.6721 - val_acc: 0.1148
Epoch 3/10
90/89 [==============================] - 276s 3s/step - loss: 1.6154 - acc: 0.5130 - val_loss: 2.7734 - val_acc: 0.1289
Epoch 4/10
90/89 [==============================] - 297s 3s/step - loss: 1.4405 - acc: 0.5954 - val_loss: 2.7180 - val_acc: 0.1289
Epoch 5/10
90/89 [==============================] - 282s 3s/step - loss: 1.3164 - acc: 0.6454 - val_loss: 2.6655 - val_acc: 0.1204
Epoch 6/10
90/89 [==============================] - 269s 3s/step - loss: 1.2058 - acc: 0.6509 - val_loss: 2.8119 - val_acc: 0.1064
Epoch 7/10
90/89 [==============================] - 291s 3s/step - loss: 1.1179 - acc: 0.6852 - val_loss: 2.6686 - val_acc: 0.1373
Epoch 8/10
90/89 [==============================] - 284s 3s/step - loss: 0.9987 - a

In [17]:
hist_mv2_l1reg = mv2_l1reg_transfer.fit_generator(
        train_generator,
        steps_per_epoch=(train_generator.samples/train_generator.batch_size),
        epochs=epochs,
        validation_data=test_generator,
        validation_steps=(test_generator.samples/test_generator.batch_size),
        verbose=1)

Epoch 1/10
30/29 [==============================] - 113s 4s/step - loss: 3.6854 - acc: 0.2591 - val_loss: 3.8041 - val_acc: 0.0952
Epoch 2/10
30/29 [==============================] - 105s 3s/step - loss: 1.8369 - acc: 0.6793 - val_loss: 4.0683 - val_acc: 0.1261
Epoch 3/10
30/29 [==============================] - 107s 4s/step - loss: 1.3457 - acc: 0.8273 - val_loss: 4.0452 - val_acc: 0.1148
Epoch 4/10
30/29 [==============================] - 109s 4s/step - loss: 1.1856 - acc: 0.8937 - val_loss: 4.3834 - val_acc: 0.1036
Epoch 5/10
30/29 [==============================] - 105s 3s/step - loss: 0.9934 - acc: 0.9409 - val_loss: 4.1332 - val_acc: 0.1176
Epoch 6/10
30/29 [==============================] - 112s 4s/step - loss: 0.9037 - acc: 0.9778 - val_loss: 4.2526 - val_acc: 0.1008
Epoch 7/10
30/29 [==============================] - 106s 4s/step - loss: 0.8534 - acc: 0.9942 - val_loss: 4.5854 - val_acc: 0.0840
Epoch 8/10
30/29 [==============================] - 108s 4s/step - loss: 0.8239 - a